<a href="https://colab.research.google.com/github/Rohith20060521/Data-Science/blob/main/Cleaning_and_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perform Transform:

✔ Cleaning Tasks
Convert "TotalCharges" to numeric (dataset has spaces → become NaN).

Fill missing numeric values using:

Median for tenure, MonthlyCharges, TotalCharges.
Replace missing categorical values with "Unknown".

✔ Feature Engineering
Create the following new columns:
1. tenure_group
Based on tenure months:
0–12   → "New"
13–36  → "Regular"
37–60  → "Loyal"
60+    → "Champion"
2. monthly_charge_segment
MonthlyCharges < 30  → "Low"
30–70              → "Medium"
> 70                 → "High"
3. has_internet_service
Convert InternetService column:
"DSL" / "Fiber optic" → 1
"No" → 0
4. is_multi_line_user
1 if MultipleLines == "Yes"
0 otherwise
5. contract_type_code
Map:
Month-to-month → 0
One year      → 1
Two year      → 2

✔ Drop unnecessary fields
Remove:
customerID, gender

In [29]:
import pandas as pd
import numpy as np

In [30]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv("/content/drive/MyDrive/WA_Fn-UseC_-Telco-Customer-Churn.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
print(df.head())
print(df.info())
print(df.shape)

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

In [32]:
df.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [33]:
df['TotalCharges']=df['TotalCharges'].replace(' ', np.nan)
df['TotalCharges']=df['TotalCharges'].astype(float)

In [34]:
df.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [35]:
df['TotalCharges']=df['TotalCharges'].fillna(df['TotalCharges'].median())

In [36]:
df.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [37]:
bins=[0,12,36,60,float('inf')]
labels=["New","Regular","Loyal","Champion"]
df['tenure_group']=pd.cut(df['tenure'],bins=bins,labels=labels)

In [38]:
df['tenure_group'].head()

,tenure_group
0,New
1,Regular
2,New
3,Loyal
4,New


In [39]:
conditions = [
    df['MonthlyCharges'] < 30,
    (df['MonthlyCharges'] >= 30) & (df['MonthlyCharges'] <= 70),
    df['MonthlyCharges'] > 70
]

choices = ["low", "medium", "high"]

df['monthly_charge_segment'] = np.select(conditions, choices,default="Unknown")

In [40]:
df['has_internet_service']=df['InternetService'].map({
    'DSL':1,
    'Fiber optic':1,
    'No':0
})

In [41]:
df['has_internet_service'].head()

,has_internet_service
0,1
1,1
2,1
3,1
4,1


In [42]:
df['is_multi_line_user']=np.where(
    df['MultipleLines']=='Yes',1,0
)

In [43]:
df['contract_type_code']=df['Contract'].map({
    'Month-to-month':0,
    'One year':1,
    'Two year':2
})

In [44]:
df.drop(columns=['customerID','gender'],inplace=True)

In [45]:
df.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,...,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group,monthly_charge_segment,has_internet_service,is_multi_line_user,contract_type_code
0,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,...,Yes,Electronic check,29.85,29.85,No,New,low,1,0,0
1,0,No,No,34,Yes,No,DSL,Yes,No,Yes,...,No,Mailed check,56.95,1889.50,No,Regular,medium,1,0,1
2,0,No,No,2,Yes,No,DSL,Yes,Yes,No,...,Yes,Mailed check,53.85,108.15,Yes,New,medium,1,0,0
3,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,...,No,Bank transfer (automatic),42.30,1840.75,No,Loyal,medium,1,0,1
4,0,No,No,2,Yes,No,Fiber optic,No,No,No,...,Yes,Electronic check,70.70,151.65,Yes,New,high,1,0,0
